<a href="https://colab.research.google.com/github/ruthreich-star/LoanDefault/blob/main/LoanDefault_Imbalanced_Data44.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import warnings
from google.colab import files
import pickle
import os
from scipy import stats
from scipy.stats import zscore

!pip install missingno
import missingno as msno

!pip install fancyimpute
import fancyimpute

!pip install --upgrade autoviz
from autoviz.AutoViz_Class import AutoViz_Class

warnings.filterwarnings("ignore")
%matplotlib inline

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 16.3 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29879 sha256=2b84b980c1e1e6cce68d42b367ffd8d00bb8fb34ca7846a300d73ef088f8d829
  Stored in directory: /root/.cache/pip/wheels/df/20/91/e4850b9a31cf660c1bc95515d3bcbc8010e869e5de6d5baf07
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11331 sha256=02308821b6e3babee633d5b8afc7cba574c02a939a60ccc7669a5f8d0413ab67
  Stored in directory: /root/.cache/pip/wheels/89/11/61/aa5a3167bfff49218cd043a73a83034b9eadd858b0059521be
Successfully built fancyimpute knnimpute
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.5/67.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
np.random.seed(31071967)

In [3]:
download_pickels = True

In [6]:
import os, pathlib, sys
!mkdir -p pickles
pickle_file = '/content/pickles/load_default_feature_eng.pkl'

if not os.path.exists(pickle_file):
# Upload file
  uploaded = files.upload()

# Read pickle into DataFrame
df = pd.read_pickle(pickle_file)

df.head(10)

,id,loan_amnt,installment,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,issue_d_month,earliest_cr_line_year,earliest_cr_line_month,last_pymnt_d_year,last_pymnt_d_month,last_credit_pull_d_year,last_credit_pull_d_month,months_between_las_p_to_issue,credit_age_months,underfunded_amt,platform_share_amt,term_num,amount_x_rate,rate_x_term,repay_fail
0,545583,2500,85.0,5,4,20004.0,9,22,19.86,5.0,7.0,0.0,981.0,21.30,10.0,7.0,2005.0,8.0,2013.0,7.0,2016.0,6.0,36,59,0,0,36,349.5,5.0328,0
1,532101,5000,176.0,5,4,59000.0,2,33,19.57,1.0,7.0,0.0,18773.0,99.90,15.0,6.0,1994.0,4.0,2011.0,11.0,2012.0,3.0,17,194,0,0,36,797.5,5.7420,1
2,877788,7000,226.0,2,0,53796.0,9,42,10.80,3.0,7.0,0.0,3269.0,47.20,20.0,9.0,1998.0,3.0,2014.0,3.0,2014.0,3.0,30,162,0,0,36,693.7,3.5676,0
3,875406,2000,60.0,2,4,30000.0,2,33,3.60,0.0,7.0,0.0,0.0,0.00,15.0,9.0,1975.0,1.0,2014.0,2.0,2016.0,6.0,29,440,0,0,36,108.4,1.9512,0
4,506439,3600,117.0,2,0,675048.0,9,1,1.55,4.0,8.0,0.0,0.0,0.00,25.0,4.0,1998.0,4.0,2013.0,5.0,2016.0,6.0,37,144,0,0,36,369.0,3.6900,0
5,981465,8000,243.0,2,0,77736.0,9,3,6.07,0.0,12.0,0.0,4182.0,13.60,49.0,10.0,1996.0,7.0,2014.0,4.0,2014.0,4.0,30,183,0,0,36,482.4,2.1708,0
6,749050,6000,187.0,4,4,35000.0,2,24,13.13,0.0,5.0,0.0,5864.0,47.70,9.0,5.0,2003.0,10.0,2014.0,5.0,2014.0,5.0,36,91,0,0,36,449.4,2.6964,0
7,1016373,25600,599.0,5,4,86000.0,2,33,26.51,1.0,16.0,0.0,33021.0,70.80,32.0,11.0,1983.0,10.0,2014.0,4.0,2016.0,6.0,29,337,127,127,60,3653.12,8.5620,0
8,786870,19750,559.0,2,0,72500.0,2,24,19.96,0.0,15.0,0.0,21544.0,98.70,44.0,6.0,1992.0,3.0,2013.0,6.0,2016.0,2.0,24,231,0,0,60,4585.95,13.9320,0
9,1019261,6250,224.0,5,0,28000.0,9,34,13.76,0.0,2.0,1.0,0.0,8.46,15.0,11.0,1999.0,10.0,2014.0,4.0,2014.0,9.0,29,145,0,0,36,1079.375,6.2172,1


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
TARGET = 'repay_fail'
def to_numeric_numpy(X: pd.DataFrame) -> pd.DataFrame:
    X = X.copy()
    # א. המרה של Int64/Float64 (nullable) ל-float32 – בטוח גם אם אין NaN
    ext = X.select_dtypes(include=['Int64', 'Float64']).columns
    X[ext] = X[ext].astype('float32')
    # ב. float64 => float32 (חיסכון בזיכרון ומהירות)
    f64 = X.select_dtypes(include=['float64']).columns
    X[f64] = X[f64].astype('float32')
    # ג. int8/int64 וכו' — להשאיר כ-int (או להפוך ל-int32 אם רוצים אחידות)
    # i32 = X.select_dtypes(include=['int64']).columns
    # X[i32] = X[i32].astype('int32')
    return X

X = df.drop(columns=[TARGET])
y = df[TARGET]

X = to_numeric_numpy(X)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38478 entries, 0 to 38477
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             38478 non-null  int64  
 1   loan_amnt                      38478 non-null  Int64  
 2   installment                    38478 non-null  float64
 3   emp_length                     38478 non-null  int8   
 4   home_ownership                 38478 non-null  int8   
 5   annual_inc                     38478 non-null  float64
 6   purpose                        38478 non-null  int8   
 7   addr_state                     38478 non-null  int8   
 8   dti                            38478 non-null  float64
 9   inq_last_6mths                 38478 non-null  float64
 10  open_acc                       38478 non-null  float64
 11  pub_rec                        38478 non-null  float64
 12  revol_bal                      38478 non-null 

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('repay_fail',axis=1), df['repay_fail'], test_size=0.3, random_state=42)

In [19]:
# Define the classification metrics function
def classificationMetrics(y, yhat):
    metrics = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }
    return metrics

In [20]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
import warnings
warnings.filterwarnings("ignore")
from xgboost import XGBClassifier

In [15]:
from xgboost import XGBClassifier
model =  XGBClassifier()

# Convert the entire dataframes to float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

#accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy: %.2f%%" % (accuracy * 100.0))
classificationMetrics(y_test, y_pred)

{'Accuracy': 0.9108627858627859,
 'Precision': 0.7768339768339768,
 'Recall': 0.5761741122565864,
 'F1-Score': 0.6616244656363038}

In [16]:
techniques = {
    "ROS": RandomOverSampler(random_state=47),
    "RUS": RandomUnderSampler(random_state=47),
    "SMOTE": SMOTE(random_state=47),
    "SMOTETomek": SMOTETomek(random_state=47)
}

In [21]:
# Evaluate and collect results using list comprehension
results = [
    {
        "Technique": name,
        "Accuracy": accuracy_score(y_test, XGBClassifier(random_state=42, eval_metric='logloss').fit(*technique.fit_resample(X_train.astype('float32'), y_train)).predict(X_test.astype('float32'))),
        "Precision": precision_score(y_test, XGBClassifier(random_state=42, eval_metric='logloss').fit(*technique.fit_resample(X_train.astype('float32'), y_train)).predict(X_test.astype('float32'))),
        "Recall": recall_score(y_test, XGBClassifier(random_state=42, eval_metric='logloss').fit(*technique.fit_resample(X_train.astype('float32'), y_train)).predict(X_test.astype('float32'))),
        "F1-Score": f1_score(y_test, XGBClassifier(random_state=42, eval_metric='logloss').fit(*technique.fit_resample(X_train.astype('float32'), y_train)).predict(X_test.astype('float32')))
    }
    for name, technique in techniques.items()
]
# Display results
results_df = pd.DataFrame(results)
print(results_df)

   Technique   Accuracy  Precision   Recall   F1-Score
0         ROS  0.894491  0.622222   0.769759  0.688172
1         RUS  0.839570  0.483078   0.866552  0.620336
2       SMOTE  0.918832  0.788311   0.633448  0.702445
3  SMOTETomek  0.919352  0.782790   0.646048  0.707876


In [22]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df

,Technique,Accuracy,Precision,Recall,F1-Score
0,ROS,0.894491,0.622222,0.769759,0.688172
1,RUS,0.839570,0.483078,0.866552,0.620336
2,SMOTE,0.918832,0.788311,0.633448,0.702445
3,SMOTETomek,0.919352,0.782790,0.646048,0.707876


The original unbalanced model achieved an Accuracy of 0.91, Precision of 0.78, Recall of 0.57, and an F1-Score of 0.66.
After applying resampling methods, the SMOTETomek technique provided the best overall results, with an Accuracy of 0.913, Precision of 0.78, Recall of 0.65, and an F1-Score of 0.71.

These results indicate a moderate but meaningful improvement in the model’s ability to correctly identify defaulted loans (higher Recall) while maintaining high overall Precision and Accuracy.
In practice, this means the model becomes more effective at detecting risky borrowers without significantly increasing false alarms.

Among all tested techniques, SMOTETomek proved to be the most balanced approach, offering the best trade-off between recall sensitivity and overall classification stability.


לאור כך הוחלט לעבור לבחינת המודל עם שיפור שהושג עי SMOTET